# 4. LangChain解説

In [ ]:
import os


os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

## 4-1 LangChainの概要

In [ ]:
!pip install --quiet langchain openai

## 4-2 Language models

### LLMs

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003", temperature=0)

result = llm("自己紹介してください。")
print(result)

### Chat models

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="こんにちは！私はジョンと言います！"),
    AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage(content= "私の名前が分かりますか？")
]

result = chat(messages)
print(result.content)

### Callbackを使ったストリーミング

In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

messages = [HumanMessage(content="自己紹介してください")]
result = chat(messages)

## 4-3 Prompts

### Prompt templates

In [ ]:
from langchain.prompts import PromptTemplate

template = """
以下の料理のレシピを教えてください。

料理名: {dish}
"""

prompt = PromptTemplate(
   input_variables=["dish"],
   template=template,
)

result = prompt.format(dish="カレー")
print(result)

### ChatPromptTemplate

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("あなたは{country}料理のプロフェッショナルです。"),
    HumanMessagePromptTemplate.from_template("以下の料理のレシピを教えてください。\n\n料理名: {dish}")
])

messages = chat_prompt.format_prompt(country="フランス", dish="肉じゃが").to_messages()

print(messages)

In [ ]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
result = chat(messages)
print(result.content)

## 4-4 Output Parsers


### PydanticOutputParser

In [ ]:
from pydantic import BaseModel, Field

class Recipe(BaseModel):
   ingredients: list[str] = Field(description="ingredients of the dish")
   steps: list[str] = Field(description="steps to make the dish")

In [ ]:
from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Recipe)

In [ ]:
from logging import Formatter
format_instructions = parser.get_format_instructions()

print(format_instructions)

In [ ]:
from langchain.prompts import PromptTemplate

template = """料理のレシピを教えてください。

{format_instructions}

料理名: {dish}
"""

prompt = PromptTemplate(
   template=template,
   input_variables=["dish"],
   partial_variables={"format_instructions": format_instructions}
)

In [ ]:
input = prompt.format(dish="カレー")

print(input)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
messages = [HumanMessage(content=input)]
output = chat(messages)

print(output.content)

In [ ]:
recipe = parser.parse(output.content)
print(recipe)

## 4-4 Chains

### LLMChain―PromptTemplate・Language model・OutputParserをつなぐ

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field

class Recipe(BaseModel):
   ingredients: list[str] = Field(description="ingredients of the dish")
   steps: list[str] = Field(description="steps to make the dish")

output_parser = PydanticOutputParser(pydantic_object=Recipe)

template = """料理のレシピを教えてください。

{format_instructions}

料理名: {dish}
"""

prompt = PromptTemplate(
   template=template,
   input_variables=["dish"],
   partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [ ]:
from langchain import LLMChain

chain = LLMChain(prompt=prompt, llm=chat, output_parser=output_parser)

recipe = chain.run(dish="カレー")

print(type(recipe))
print(recipe)

### SimpleSequentialChain―ChainとChainをつなぐ

In [ ]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

cot_template = """以下の質問に回答してください。

質問: {question}

ステップバイステップで考えましょう。
"""

cot_prompt = PromptTemplate(
   input_variables=["question"],
   template=cot_template,
)

cot_chain = LLMChain(llm=chat, prompt=cot_prompt)

In [ ]:
summarize_template = """以下の文章を結論だけ一言に要約してください。

{input}
"""
summarize_prompt = PromptTemplate(
   input_variables=["input"],
   template=summarize_template,
)

summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)

In [ ]:
from langchain.chains import SimpleSequentialChain

cot_summarize_chain = SimpleSequentialChain(chains=[cot_chain, summarize_chain])

result = cot_summarize_chain(
   "私は市場に行って10個のリンゴを買いました。隣人に2つ、修理工に2つ渡しました。それから5つのリンゴを買って1つ食べました。残りは何個ですか？")
print(result["output"])

## 4-5 Memory

### ConversationBufferMemory

In [ ]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

import langchain
langchain.debug = True
langchain.verbose = False

chat = ChatOpenAI(model_name="gpt-4", temperature=0)
conversation = ConversationChain(
    llm=chat,
    memory=ConversationBufferMemory()
)

while True:
    user_message = input("You: ")
    ai_message = conversation.run(input=user_message)
    print(f"AI: {ai_message}")

In [ ]:

from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="こんにちは！私はジョンと言います！"),
    AIMessage(content="こんにちは、ジョンさん！どのようにお手伝いできますか？"),
    HumanMessage(content= "私の名前が分かりますか？")
]

result = chat(messages)
print(result.content)

## 4-7 Data connection

### Document loaders

In [ ]:
!pip install GitPython

In [ ]:
!rm -rf ./langchain

In [ ]:
from langchain.document_loaders import GitLoader

def file_filter(file_path):
  return file_path.endswith(".mdx")

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

raw_docs = loader.load()
print(len(raw_docs))

### Document transformers

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(raw_docs)
print(len(docs))

## Text embedding models

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [ ]:
query = "AWSのS3からデータを読み込むためのDocumentLoaderはありますか？"

vector = embeddings.embed_query(query)
print(len(vector))
print(vector)

### Vector stores

In [ ]:
!pip install --quiet chromadb tiktoken

In [ ]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(docs, embeddings)

### Retriever

In [ ]:
retriever = db.as_retriever()

In [ ]:
query = "AWSのS3からデータを読み込むためのDocumentLoaderはありますか？"

context_docs = retriever.get_relevant_documents(query)

print(len(context_docs))

first_doc = context_docs[0]
print(first_doc.metadata)
print(first_doc.page_content)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm=chat, chain_type="stuff", retriever=retriever)

result = qa_chain.run(query)
print(result)

## 4-8 Agents

### Agentsの使用例

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI

import langchain
langchain.verbose = True
langchain.debug = False

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
tools = load_tools(["terminal"], llm=chat)
agent_chain = initialize_agent(
    tools, chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

result = agent_chain.run("sample_dataディレクトリにあるファイルの一覧を教えて")
print(result)

### Tools

In [ ]:
from langchain.tools import Tool

def my_super_func(input):
  return "42"

tools = [
    Tool.from_super_function(
        func=my_func,
        name="The_Answer",
        description="生命、宇宙、そして万物についての究極の疑問の答え"
    ),
]

In [ ]:
langchain.verbose = True
langchain.debug = False

agent_chain = initialize_agent(
    tools, chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

result = agent_chain.run("この世界の真理を教えてください")
print(result)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain import LLMChain

summarize_template = """以下の文章を結論だけ一言に要約してください。

{input}
"""
summarize_prompt = PromptTemplate(
   input_variables=["input"],
   template=summarize_template,
)

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)

tools = [
    Tool.from_function(
        func=summarize_chain.run,
        name="Summarizer",
        description="Text summarizer"
    ),
]


In [ ]:
langchain.verbose = True
langchain.debug = False

agent_chain = initialize_agent(
    tools, chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

input = """以下を要約してください。

こんにちは！私はChatGPTと呼ばれるAI言語モデルです。OpenAIが開発したGPT-3.5アーキテクチャに基づいています。私は自然言語理解と生成に特化しており、さまざまなトピックに関する質問に答えたり、おしゃべりしたりすることが得意です。

私のトレーニングデータは2021年9月までの情報に基づいているため、それ以降の出来事については知識がありません。ですが、できる限りお手伝いすることに努めます。

質問や会話、情報の共有など、どんなお手伝いでもお気軽にお申し付けください！よろしくお願いします。"""

result = agent_chain.run(input)
print(result)

In [ ]:
from langchain.agents.agent_toolkits import GmailToolkit

toolkit = GmailToolkit()
tools = toolkit.get_tools()
print(tools)

In [ ]:
!pip install pygithub

In [ ]:
import os
from langchain.agents.agent_toolkits.github.toolkit import GitHubToolkit
from langchain.utilities.github import GitHubAPIWrapper

os.environ["GITHUB_APP_ID"] = "your-github-app-id"
os.environ["GITHUB_APP_PRIVATE_KEY"] = "/path/to/your/private/key"
os.environ["GITHUB_REPOSITORY"] = "user/repo"
os.environ["GITHUB_BRANCH"] = "branch-name"

github = GitHubAPIWrapper()
toolkit = GitHubToolkit.from_github_api_wrapper(github)
tools = toolkit.get_tools()
print(tools)

### Function callingを使うOpenAI Functions Agent

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
tools = load_tools(["terminal"], llm=chat)
agent_chain = initialize_agent(tools, chat, agent=AgentType.OPENAI_FUNCTIONS)

result = agent_chain.run("sample_dataディレクトリにあるファイルの一覧を教えて")
print(result)

### 複数のツールを一度に使うOpenAI Multi Functions Agent

In [ ]:
!pip install --quiet duckduckgo-search

In [ ]:
import langchain
import openai

langchain.debug = True
langchain.verbose = False
openai.log = "info"

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
tools = load_tools(["ddg-search"])
agent_chain = initialize_agent(tools, chat, agent=AgentType.OPENAI_MULTI_FUNCTIONS)

result = agent_chain.run("東京と大阪の天気を教えてください")
print(result)

In [ ]:
from langchain.callbacks import HumanApprovalCallbackHandler
from langchain.tools import ShellTool

tool = ShellTool(callbacks=[HumanApprovalCallbackHandler()])
print(tool.run("echo Hello World!"))

### （コラム）Function callingを応用したOurputParser・Extraction・Tagging

In [ ]:
import json

from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain, create_extraction_chain_pydantic
from langchain.prompts import ChatPromptTemplate

schema = {
    "properties": {
        "person_name": {"type": "string"},
        "person_height": {"type": "integer"},
        "person_hair_color": {"type": "string"},
        "dog_name": {"type": "string"},
        "dog_breed": {"type": "string"},
    },
    "required": ["person_name", "person_height"],
}
input = """
Alex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde.
Alex's dog Frosty is a labrador and likes to play hide and seek.
"""

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_extraction_chain(schema, chat)

people = chain.run(input)
print(json.dumps(people, indent=2))

## まとめ

### （コラム）Evaluation

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.evaluation import load_evaluator

chat = ChatOpenAI(model="gpt-4", temperature=0)

evaluator = load_evaluator("qa", eval_llm=chat)

result = evaluator.evaluate_strings(
    input="私は市場に行って10個のリンゴを買いました。隣人に2つ、修理工に2つ渡しました。それから5つのリンゴを買って1つ食べました。残りは何個ですか？",
    prediction="""1最初に10個のリンゴを買い、その中から隣人と修理工にそれぞれ2個ずつ渡しました。そのため、まず手元に残ったリンゴは10 - 2 - 2 = 6個となります。

その後、さらに5個のリンゴを買い、1つ食べました。これにより手元のリンゴは6 + 5 - 1 = 10個となります。""",
    reference="10個",
)

print(result)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.evaluation import load_evaluator

llm = ChatOpenAI(model="gpt-4", temperature=0)

evaluator = load_evaluator("qa", eval_llm=llm)

result = evaluator.evaluate_strings(
    input="パンはパンでも食べられないパンは？",
    prediction="フライパン",
    reference="フライパン",
)

print(result)